# Create a custom OBB table

In this tutorial, we will create a custom 3LC Table with oriented bounding boxes from the [HRSC2016-MS](https://github.com/wmchen/HRSC2016-MS) dataset
first introduced in the paper `MSSDet: Multi-Scale Ship-Detection Framework in Optical Remote-Sensing Images and New Benchmark`.

The dataset contains 1070 images with 4406 annotated ships.

## Project setup

In [ ]:
PROJECT_NAME = "3LC Tutorials"
DATASET_NAME = "HRSC2016-MS"

## Imports

In [ ]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from pathlib import Path

import tlc

## Prepare the data

The data can be downloaded as a 2.3 GB zip file from the dataset's [GitHub repository](https://github.com/wmchen/HRSC2016-MS).
When unzipped, it has the folder structure:

```
HRSC2016-MS/
├── Annotations/
├── AllImages/
└── ImageSets/
```

In [ ]:
DATASET_ROOT = "D:/Data/HRSC2016-MS"  # Replace with your local path

# Register the dataset root as a URL alias to make it easier to share the tables and move the source data.
tlc.register_project_url_alias("HRSC2016_MS_DATA", DATASET_ROOT, project=PROJECT_NAME)

In [ ]:
row_data = defaultdict(dict)

for split in ["train", "val"]:
    image_splits = Path(DATASET_ROOT) / "ImageSets" / f"{split}.txt"
    image_ids = image_splits.read_text().splitlines()
    row_data[split] = defaultdict(list)

    for image_id in image_ids:
        image_path = Path(DATASET_ROOT) / "AllImages" / f"{image_id}.bmp"
        annotation_path = Path(DATASET_ROOT) / "Annotations" / f"{image_id}.xml"
        if not image_path.exists():
            print(f"Image {image_id} does not exist")
        if not annotation_path.exists():
            print(f"Annotation {image_id} does not exist")
        row_data[split]["image"].append(tlc.Url(image_path).to_relative().to_str())
        row_data[split]["obb"].append(annotation_path)

In [ ]:
def load_obb_annotation(annotation_path):
    """Load annotations for a single image from XML format."""
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)
    obbs = tlc.OBBInstances.create_empty(image_height=height, image_width=width)
    for obj in root.findall("object"):
        difficult = int(obj.find("difficult").text)
        truncated = int(obj.find("truncated").text)
        bbox = obj.find("robndbox")
        cx, cy, w, h, angle = (float(bbox.find(tag).text) for tag in ["cx", "cy", "w", "h", "angle"])
        obbs.add_instance(
            obb=[cx, cy, w, h, angle],
            label=0,  # single class dataset—all instances are ships
            extras={
                "difficult": difficult,
                "truncated": truncated,
            },
        )

    return obbs.to_row()

In [ ]:
for split in ["train", "val"]:
    row_data[split]["obb"] = [load_obb_annotation(path) for path in row_data[split]["obb"]]

## Create the tables

We use a `Geometry2DSchema` to describe the structure of the oriented bounding boxes, and `tlc.Table.from_dict` to create the tables.

In [ ]:
schemas = {
    "image": tlc.Schema(value=tlc.ImageUrlStringValue()),
    "obb": tlc.OrientedBoundingBoxes2DSchema(
        classes=["ship"],
        per_instance_schemas={
            "difficult": tlc.BoolListSchema(),
            "truncated": tlc.BoolListSchema(),
        },
    ),
}

In [ ]:
for split in ["train", "val"]:
    table = tlc.Table.from_dict(
        data=row_data[split],
        structure=schemas,
        table_name=f"{split}",
        dataset_name=DATASET_NAME,
        project_name=PROJECT_NAME,
        if_exists="rename",
    )

In [ ]:
table